In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Utility import spawn_camera, spawn_vehicle, preprocess_image, controllo_velocita

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [25]:
#macchina spawnata tipo 1
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-2, -41.3, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))


In [26]:
# Configura i controlli iniziali
control = carla.VehicleControl()
control.reverse = False  # Retromarcia
target_speed_mps = 10 / 3.6

# Passi per il parcheggio
PARKING_STEP_0_DISTANCE = 1.3
PARKING_STEP_1_DISTANCE = 1.8  # Tempo per sterzare all'indietro
PARKING_STEP_2_DISTANCE = 1.5  # Tempo per raddrizzare

distance_travelled = 0.0

try:
    control.steer = -0.5  # Sterzata a destra (positivo per destra)
    while distance_travelled < PARKING_STEP_0_DISTANCE:
        current_velocity = vehicle.get_velocity()
        current_speed_mps = current_velocity.length()
        distance_travelled += current_speed_mps * 0.05
        control = controllo_velocita(control, target_speed_mps, current_speed_mps)
        vehicle.apply_control(control)
        time.sleep(0.1)

    # Step 1: Sterzata iniziale per entrare nel parcheggio
    control.steer = 0.5  # Sterzata a destra (positivo per destra)
    control.reverse = True
    distance_travelled = 0.0
    while distance_travelled < PARKING_STEP_1_DISTANCE:
        current_velocity = vehicle.get_velocity()
        current_speed_mps = current_velocity.length()
        distance_travelled += current_speed_mps * 0.05
        control = controllo_velocita(control, target_speed_mps, current_speed_mps)
        vehicle.apply_control(control)
        time.sleep(0.1)

    # Step 3: Retromarcia diritta per completare il parcheggio
    control.steer = 0.0  # Sterzo neutro
    distance_travelled = 0.0
    while distance_travelled < PARKING_STEP_2_DISTANCE:
        current_velocity = vehicle.get_velocity()
        current_speed_mps = current_velocity.length()
        distance_travelled += current_speed_mps * 0.05
        control = controllo_velocita(control, target_speed_mps, current_speed_mps)
        vehicle.apply_control(control)
        time.sleep(0.1)

    # Ferma il veicolo
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)

    print("Parcheggio completato!")

    time.sleep(3)

finally:
    # Despawn del veicolo
    if vehicle is not None:
        print("Distruggendo il veicolo...")
        vehicle.destroy()
        print("Veicolo rimosso.")

Parcheggio completato!
Distruggendo il veicolo...
Veicolo rimosso.
